## Accessing DM-EFD data


In this notebook we demonstrate how to extract data from the DM-EFD using [aioinflux](https://aioinflux.readthedocs.io/en/stable/index.html), a Python client for InfluxDB, and proceed with data analysis using Pandas dataframes. 

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

In addition to `aioinflux`, you'll need to install `pandas`, `numpy` and `matplotlib` to run this notebook.

In [ ]:
import matplotlib
%matplotlib widget
from matplotlib import pylab as plt
import aioinflux
import getpass
import pandas as pd
import asyncio
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
output_notebook()

We'll access the DM-EFD instance deployed at the AuxTel lab in Tucson. You need to be on site or connected to the NOAO VPN. 

If you are familiar with the AuxTel lab environment, you might be able to authenticate using the generic `saluser`. Ping me at Slack (`@afausti`) if you have any problem.

In [ ]:
username = "saluser"
password = getpass.getpass(f"Password for {username}:")

The following configures the `aioinflux` Python client to connect to the DM-EFD InfluxDB instance. 

In [ ]:
client = aioinflux.InfluxDBClient(host='summit-influxdb-efd.lsst.codes', 
                                  port='443', 
                                  ssl=True, 
                                  username=username, 
                                  password=password,
                                  db='efd')

We can configure the output to be a Pandas dataframe, which is very convenient for data analysis.  Specify a time range for data in `InfluxQL`.  The default is 20hrs ago, but this may need to be changed depending on how recently data was taken.

In [ ]:
client.output = 'dataframe'
#time_span = "time >= '2019-09-08T01:41:00Z' AND time <= '2019-09-08T01:44:00Z'"
#time_span = "time >= '2019-09-18T00:47:00Z' AND time <= '2019-09-19T04:49:00Z'"
time_span = "time >= '2019-09-20T04:00:00Z' AND time <= '2019-09-20T04:04:00Z'"
time_span = "time >= '2019-09-20T03:55:00Z' AND time <= '2019-09-20T04:02:00Z'"
# Had a stop tracking issue occur at 9:49pm

Query each of the measurements we may want to correlate later in the notebook.  Note that this could be done as a single query, but the result is a dictionary of `DataFrames` which I find less convenient to use than named variables corresponding to one `DataFrame` each.

In [ ]:
async def get_data_frame(field_base, topic, els=100):
    fields = ", ".join([f'"{field_base}{i}"' for i in range(100)])
    df = await client.query(f'SELECT "cRIO_timestamp", "private_sndStamp", {fields} FROM "efd"."autogen"."{topic}" WHERE {time_span}')

    times = []
    timestamps = []
    vals = []
    step = 1./els
    for row in df.itertuples():
        for i in range(els):
            times.append(row.cRIO_timestamp + i*step)
            timestamps.append((pd.Timestamp(row.cRIO_timestamp, unit='s', tz='GMT') + pd.Timedelta(i*step, unit='s')))
            vals.append(getattr(row, f'{field_base}{i}'))

    return pd.DataFrame({'times':times, field_base:vals}, index=timestamps)

In [ ]:
az = await get_data_frame('azimuthCalculatedAngle', 'lsst.sal.ATMCS.mount_AzEl_Encoders')
az1 = await get_data_frame('azimuthEncoder1Raw', 'lsst.sal.ATMCS.mount_AzEl_Encoders')
az2 = await get_data_frame('azimuthEncoder2Raw', 'lsst.sal.ATMCS.mount_AzEl_Encoders')
az3 = await get_data_frame('azimuthEncoder3Raw', 'lsst.sal.ATMCS.mount_AzEl_Encoders')
az_raw = await client.query(f'SELECT "azimuthCalculatedAngle99", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.mount_AzEl_Encoders" WHERE {time_span}')

In [ ]:
commanded_az = await get_data_frame('azimuth', 'lsst.sal.ATMCS.trajectory')
commanded_az_raw = await client.query(f'SELECT "azimuth99", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.trajectory" WHERE {time_span}')

In [ ]:
#commanded_az_ATPng = await get_data_frame('azimuth', 'lsst.sal.ATMCS.command_trackTarget')
commanded_az_ATPng = await client.query(f'SELECT "azimuth", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')
#commanded_vel_az = await get_data_frame('azimuthVelocity', 'lsst.sal.ATMCS.trajectory')

In [ ]:
print(commanded_az_ATPng['private_sndStamp'][0:2])
#print(commanded_az.index.values[0])
print(commanded_az_raw['private_sndStamp'][0:2])

In [ ]:
print(commanded_az_ATPng['private_rcvStamp'][0:2])
#print(commanded_az.index.values[0])
print(commanded_az_raw['private_rcvStamp'][0:2]) # from ATMCS target

In [ ]:
pd.to_datetime(commanded_az_ATPng['private_sndStamp'][0], unit='s')

## Look at difference in timestamps that come from SAL

In [ ]:
# print(commanded_az_ATPng['private_sndStamp'][0:8]-commanded_az_ATPng['private_rcvStamp'][0:8])

# data = commanded_az_ATPng['private_sndStamp']-commanded_az_ATPng['private_rcvStamp']
# hist, edges = np.histogram(data, density=True, bins=200)

# p = figure(title='Title', tools='', background_fill_color="#fafafa", x_range=(edges[0], edges[-1]))
# p.quad(top=hist*data.size, bottom=0,  left=edges[:-1], right=edges[1:],
#        fill_color="navy", line_color="white", alpha=0.5)

# p.y_range.start = 0
# p.xaxis.axis_label = 'private_sndStamp - private_Stamp'
# p.yaxis.axis_label = 'Number'
# p.grid.grid_line_color="white"
# show(p)

In [ ]:
#data.min(), data.max(), data.median()

In [ ]:
#hist*len(data)

## Plot commanded position (by pointing component), target position by ATMCS, Calculated position by ATMCS

In [ ]:
dy=1*1/3600
yr_cen=np.median(commanded_az['azimuth'])
#yr_cen=0
dy=2.2
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_az.index.values, y=commanded_az['azimuth'], color='black', line_width=2, legend='Commanded Az')
p.cross(x=az.index.values, y=az['azimuthCalculatedAngle'], color='red', line_width=2, line_dash='dashed', legend='Computed Az 100Hz')
p.cross(x=pd.to_datetime(commanded_az_raw['private_sndStamp'], unit='s'), y=commanded_az_raw['azimuth99'], color='black', line_width=2, legend='ATMCS Target Az')
p.cross(x=pd.to_datetime(commanded_az_ATPng['private_sndStamp'], unit='s'), y=commanded_az_ATPng['azimuth'], color='green', line_width=2, legend='ATPng Target Az')
p.line(x=pd.to_datetime(az_raw['private_sndStamp'], unit='s'), y=az_raw['azimuthCalculatedAngle99'], color='lightblue', line_width=2, legend='Computed Az')


p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Now look at raw encoder counts to see what that disconitunity is...

In [ ]:
dy=1*1/3600
yr_cen=np.median(commanded_az['azimuth'])
#yr_cen=0
dy=2.2
p = figure(x_axis_type='datetime', y_range=(-1e6, 1e6), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Raw Encoder cts - median + X000 offset"
p.xaxis.axis_label = "Time"


#p.extra_y_ranges = {'encoders': Range1d(start=-1e6, end=1e6)} #Range1d(start=-0.3, end=0.2)

print ('az1 [min,max] = [{},{}]'.format(az1['azimuthEncoder1Raw'].min()-az1['azimuthEncoder1Raw'].median(), az1['azimuthEncoder1Raw'].max()-az1['azimuthEncoder1Raw'].median()))
p.cross(x=az1.index.values, y=az1['azimuthEncoder1Raw']-az1['azimuthEncoder1Raw'].median(), color='red', alpha=0.5, legend='AzEncoder 1')
p.line(x=az1.index.values, y=az1['azimuthEncoder1Raw']-az1['azimuthEncoder1Raw'].median(), color='red', alpha=0.7, legend='AzEncoder 1')

print ('az2 [min,max] = [{},{}]'.format(az2['azimuthEncoder2Raw'].min()-az2['azimuthEncoder2Raw'].median(), az2['azimuthEncoder2Raw'].max()-az2['azimuthEncoder2Raw'].median()))
p.cross(x=az2.index.values, y=az2['azimuthEncoder2Raw']-az2['azimuthEncoder2Raw'].median()+1000, color='green', alpha=0.5, legend='AzEncoder 2 +1000 offset')
p.line(x=az2.index.values, y=az2['azimuthEncoder2Raw']-az2['azimuthEncoder2Raw'].median()+1000, color='green', alpha=0.7, legend='AzEncoder 2 +1000 offset')

print ('az3 [min,max] = [{},{}]'.format(az3['azimuthEncoder3Raw'].min()-az3['azimuthEncoder3Raw'].median(), az3['azimuthEncoder3Raw'].max()-az3['azimuthEncoder3Raw'].median()))
p.cross(x=az3.index.values, y=az3['azimuthEncoder3Raw']-az3['azimuthEncoder3Raw'].median()-1000, color='blue', alpha=0.5, legend='AzEncoder 3 -1000 offset')
p.line(x=az3.index.values, y=az3['azimuthEncoder3Raw']-az3['azimuthEncoder3Raw'].median()-1000, color='blue', alpha=0.7, legend='AzEncoder 3 -1000 offset')


p.legend.location = 'bottom_right'
p.legend.click_policy = 'hide'
show(p)

## Plot the ratios of the encoder counts to see if gaps/slopes are different

In [ ]:
p = figure(x_axis_type='datetime', plot_width=800, plot_height=400)
p.yaxis.axis_label = "Raw Encoder counts - median value"
p.xaxis.axis_label = "Time"

#p.add_layout(LinearAxis(y_range_name='encoders', axis_label='Encoder Counts'), 'right')

median31=np.median(az3['azimuthEncoder3Raw']/az1['azimuthEncoder1Raw'])
p.cross(x=az3.index.values, y=az3['azimuthEncoder3Raw']/az1['azimuthEncoder1Raw']-median31, color='red', alpha=0.5, legend='AzEncoder 3/1')
p.line(x=az3.index.values, y=az3['azimuthEncoder3Raw']/az1['azimuthEncoder1Raw']-median31, color='red', alpha=0.7, legend='AzEncoder 3/1')

median21=np.median(az2['azimuthEncoder2Raw']/az1['azimuthEncoder1Raw'])
p.cross(x=az2.index.values, y=az2['azimuthEncoder2Raw']/az1['azimuthEncoder1Raw']-median21, color='green', alpha=0.5, legend='AzEncoder 2/1')
p.line(x=az2.index.values, y=az2['azimuthEncoder2Raw']/az1['azimuthEncoder1Raw']-median21, color='green', alpha=0.7, legend='AzEncoder 2/1')

median32=np.median(az3['azimuthEncoder3Raw']/az2['azimuthEncoder2Raw'])
p.cross(x=az2.index.values, y=az3['azimuthEncoder3Raw']/az2['azimuthEncoder2Raw']-median32, color='blue', alpha=0.5, legend='AzEncoder 3/2')
p.line(x=az2.index.values, y=az3['azimuthEncoder3Raw']/az2['azimuthEncoder2Raw']-median32, color='blue', alpha=0.7, legend='AzEncoder 3/2')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
az1['azimuthEncoder1Raw'].median(), az2['azimuthEncoder2Raw'].median(), az3['azimuthEncoder3Raw'].median()

## Look at motor velocities in azimuth

In [ ]:
#Measured
measured_vel_az1 = await get_data_frame('azimuthMotor1Velocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
measured_vel_az2 = await get_data_frame('azimuthMotor2Velocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
# Commanded
commanded_az = await get_data_frame('azimuth', 'lsst.sal.ATMCS.trajectory')
commanded_vel_az = await get_data_frame('azimuthVelocity', 'lsst.sal.ATMCS.trajectory')

In [ ]:
p = figure(x_axis_type='datetime', y_range=(261., 261.4), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth (degrees)"
p.xaxis.axis_label = "Time"
# Positions
p.line(x=commanded_az.index.values, y=commanded_az['azimuth'], color='black', line_width=2, legend='Commanded Az Vel')
p.line(x=az.index.values, y=az['azimuthCalculatedAngle'], color='lightblue', line_width=2, legend='Computed Az Vel')
# Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.line(x=commanded_vel_az.index.values, y=commanded_vel_az['azimuthVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend='Commanded Az Velocity')
p.line(x=measured_vel_az1.index.values, y=measured_vel_az1['azimuthMotor1Velocity'], color='red', alpha=0.5, y_range_name='Velocity', legend='Measured Az Velocity Motor 1')
p.line(x=measured_vel_az2.index.values, y=measured_vel_az2['azimuthMotor2Velocity'], color='blue', alpha=0.5, y_range_name='Velocity', legend='Measured Az Velocity Motor 2')
p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

## Now look at the Elevation Axis

In [ ]:
el = await get_data_frame('elevationCalculatedAngle', 'lsst.sal.ATMCS.mount_AzEl_Encoders')

In [ ]:
# Looks like demanded Torques are too small to get measured...
#el_motor_torque_demand = await get_data_frame('elevationMotorTorque', 'lsst.sal.ATMCS.ATMCS_torqueDemand')
#el_motor_torque_measured = await get_data_frame('elevationMotorTorque', 'lsst.sal.ATMCS.ATMCS_measuredTorque')

In [ ]:
measured_vel_el = await get_data_frame('elevationMotorVelocity', 'lsst.sal.ATMCS.measuredMotorVelocity')
#measured_vel_el.to_csv(path_or_buf='/home/patrickingraham/DATA/test.csv', index=True)

In [ ]:
commanded_el = await get_data_frame('elevation', 'lsst.sal.ATMCS.trajectory')
commanded_vel_el = await get_data_frame('elevationVelocity', 'lsst.sal.ATMCS.trajectory')

In [ ]:
commanded_el_ATPng = await client.query(f'SELECT "elevation", "private_sndStamp", "private_rcvStamp" FROM "efd"."autogen"."lsst.sal.ATMCS.command_trackTarget" WHERE {time_span}')

In [ ]:
#commanded_el_ATPng

In [ ]:
p = figure(x_axis_type='datetime', y_range=(56,57 ), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Elevation (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_el.index.values, y=commanded_el['elevation'], color='black', line_width=2, legend='Commanded Elevation')
p.line(x=el.index.values, y=el['elevationCalculatedAngle'], color='lightblue', line_width=2, legend='Computed Elevation')
p.cross(x=pd.to_datetime(commanded_el_ATPng['private_sndStamp'], unit='s'), y=commanded_el_ATPng['elevation'], color='green', line_width=2, legend='ATPng Target El')
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity'), 'right')
p.cross(x=measured_vel_el.index.values, y=measured_vel_el['elevationMotorVelocity'], color='red', alpha=0.5, y_range_name='Velocity', legend='Measured El Velocity Motor')
p.line(x=commanded_vel_el.index.values, y=commanded_vel_el['elevationVelocity'], color='black', alpha=0.5, y_range_name='Velocity', legend='Commanded El Velocity')
p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

In [ ]:
p = figure(x_axis_type='datetime', y_range=(-2e-4,2e-4), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Commanded - measured Elevation (degrees)"
p.xaxis.axis_label = "Time"
p.line(x=commanded_el.index.values, y=commanded_el['elevation']-el['elevationCalculatedAngle'], color='black', line_width=2, legend='Commanded Elevation')
p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)